### run_main.py

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))

from python_scripts.api_calls import fetch_item_from_api, fetch_item_to_df, fetch_items, get_cookie_from_blob
from make_graphs import plot_historic_price, plot_historic_volume,  plot_lognormalised_historic_price, plot_lognormalised_historic_volume, plot_simple_moving_average, plot_exponential_moving_average, plot_weighted_least_squares, plot_bollinger_bands, plot_relative_strength_index, plot_money_flow_index
from python_scripts.calculate_metrics import create_ln_df, calculate_sma, calculate_ema, calculate_bollinger_bands, calculate_price_percentage_change, calculate_relative_strength_index, calculate_market_cap, calculate_money_flow_index, calculate_market_cap_jupyter

dailyCookie = get_cookie_from_blob()
items = fetch_items()

current_item = fetch_item_to_df(items[5], dailyCookie)
non_aggregated_item = fetch_item_from_api(items[5], dailyCookie)
print(items[5])

print(current_item.tail())
print(non_aggregated_item.tail())

START_DATE = current_item.index[0]
END_DATE = current_item.index[-1]




### historic_price
 

In [ ]:
#outputs historic price

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

df = current_item.copy()


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


interact(lambda start_date, end_date: plot_historic_price(df, start_date, end_date),
         start_date=start_date_picker, end_date=end_date_picker)

### historic_volume

In [ ]:
# > Volume Indicatior (1)

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

df = current_item.copy()


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


interact(lambda start_date, end_date: plot_historic_volume(df,  start_date, end_date),
         start_date=start_date_picker, end_date=end_date_picker)

### lognormal_historic_data

In [ ]:
#outputs log normalised historic price + volume charts

import pandas as pd
import numpy as np
from ipywidgets import interact, DatePicker
import plotly.graph_objects as go

ln_df = create_ln_df(current_item)

start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

#NOTE: Top is LN price, bottom in LN volume.

interact(lambda start_date, end_date: plot_lognormalised_historic_price(ln_df, start_date, end_date),
         start_date=start_date_picker, end_date=end_date_picker)

interact(lambda start_date, end_date: plot_lognormalised_historic_volume(ln_df, start_date, end_date),
         start_date=start_date_picker, end_date=end_date_picker)



### simple_moving_average

In [ ]:
# SIMPLE Moving Average (SMA)                            


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()
# NOTE: Window = sets the period for calculating the moving average.
window = 2
df = calculate_sma(df, window)


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


interact(lambda start_date, end_date:
            plot_simple_moving_average(df, start_date, end_date),
            start_date=start_date_picker, end_date=end_date_picker)



### exponential_moving_average

In [ ]:
 # Exponential Moving Average (EMA)  
 # | Part of Moving Averages (2) block

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()

# NOTE: Span = Used to calculate the decay factor. Higher span -> greater exponential smoothing
span = 10
df = calculate_ema(df, span)

start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


interact(lambda start_date, end_date:
            plot_exponential_moving_average(df, start_date, end_date),
            start_date=start_date_picker, end_date=end_date_picker)

### ordinary_least_squares & weighted_least_squares (ML)

In [ ]:
# Weighted Least Squares Estimator (3.2) - 
# Note: this makes OLS (above) redundant since you can adjust weighting to 1 in the 'weight' array

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()

# Weights = determine the influence of data points in WLS; using weights of 1,1 yields OLS results (OLS has uniform weightings).

df['date_ordinal'] = df.index.map(pd.Timestamp.toordinal)
X = df['date_ordinal'].values.reshape(-1, 1)
y = df['price_usd'].values

weights = np.linspace(1, 10, len(X))

model_wls = sm.WLS(y, sm.add_constant(X), weights=weights).fit()
y_pred = model_wls.predict(sm.add_constant(X))

split_point = int(len(X) * 0.80)
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]





start_date_picker = DatePicker(description='Start Date', value=df.index[0], disabled=False)
end_date_picker = DatePicker(description='End Date', value=df.index[-1], disabled=False)


interact(lambda start_date, end_date:
            plot_weighted_least_squares(df, start_date, end_date, y_pred, split_point, y_test),
            start_date=start_date_picker, end_date=end_date_picker)





### bollinger_bands

In [ ]:
# # Bollinger Bands - Bolligner Bands currently use the SMA for the upper + lower band, but standardised to historical price                           


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()

# Window sets the period length for calculating the Bollinger Bands moving average. Industry standard is 20d. 
window = 20
df= calculate_bollinger_bands(df, window)


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

interact(lambda start_date, end_date:
            plot_bollinger_bands(df, start_date, end_date),
            start_date=start_date_picker, end_date=end_date_picker)

### price_percentage_change

In [ ]:
# Percentage Change output

from ipywidgets import DatePicker, interact

df = current_item.copy()

start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


def update_percentage_change(start_date, end_date):
    calculate_price_percentage_change(df, start_date, end_date)

interact(update_percentage_change, start_date=start_date_picker, end_date=end_date_picker)


### relative_strength_index

In [ ]:
#rsi 

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

 
df = non_aggregated_item.copy()

window = "5d"
rsi_data = calculate_relative_strength_index(df, window)


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

interact(lambda start_date, end_date:
            plot_relative_strength_index(start_date, end_date, rsi_data, window),
            start_date=start_date_picker, end_date=end_date_picker)



### money_flow_index

In [ ]:
#mfi

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# Copy the data
df = non_aggregated_item.copy()

# Define window for resampling
window = "5d"
mfi_data = calculate_money_flow_index(df, window)

# Date pickers for interaction
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

interact(lambda start_date, end_date:
            plot_money_flow_index(start_date, end_date, mfi_data, window),
            start_date=start_date_picker, end_date=end_date_picker)



### market_cap

In [ ]:
# Market Cap Calculation - for verification purposes/single date

df = non_aggregated_item.copy()

market_cap = df.resample('d').agg({
 
    'price_usd': ['last'],
    'volume': 'sum'
      })
     # Slices the multi-level columns and renames
market_cap.columns = ['close','volume']



def update_market_cap(start_date, end_date):
     calculate_market_cap_jupyter(market_cap, start_date, end_date)

start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

interact(lambda start_date, end_date: calculate_market_cap_jupyter(market_cap, start_date, end_date),
         start_date=start_date_picker, end_date=end_date_picker)

### CANDLESTICK - NOT DONE YET - PLAN: IMPLIMENT IN EVERY CODEBLOCK AS TRUE/FALSE FUNCTION

In [ ]:
#CANDLESTICK BUT WITH FIRST AND LAST INSTEAD OF MEAN AND MEDIAN

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item.copy()

# Resample the data to weekly timeframe
# NOTE: Range can be altered: 'W' (week), 'M' (Month), '2D' (2 Days)
new_data = df.resample('w').agg({
    'price_usd': ['max', 'min', 'first', 'last'],
    'volume': 'sum'
})


# Slices the multi-level columns and renames
new_data.columns = ['high', 'low', 'open', 'close' ,'volume']


# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):
 

    mask = (new_data.index >= pd.to_datetime(start_date)) & (new_data.index <= pd.to_datetime(end_date))
    filtered_data = new_data.loc[mask]

    
    # Creating the candlestick chart
    fig = go.Figure(data=[go.Candlestick(x=filtered_data.index,
                                         open=filtered_data['open'],
                                         high=filtered_data['high'],
                                         low=filtered_data['low'],
                                         close=filtered_data['close'],
                                         name='Candlestick')])
    


    # Formatting
    fig.update_layout(
        title='SMA - Candlestick',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        
        showlegend=True
    )

    fig.show()




# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget

interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)